In [56]:
import sys
sys.path.insert(0, "/home/aisummer/ssemenov_workspace/nlp-service")

import pandas as pd 
import ast
import numpy as np
import json
import os
import nltk

from src.DocumentsParser.utils import DBS_DIR_DENSE_VECTORDB_NAME, DBS_DIR_SPARSE_VECTORDB_NAME
from src.evaluation_metrics import ReaderMetrics
from src.DocumentsSummarizer.Summarizer import SummarizerModule
from src.LLM_build.utils import llmConfig
from src.utils import DialogueState
from src.logger import Logger

### 1) Создаём объекты

In [57]:
reader = SummarizerModule()
config = DialogueState()
configLLM = llmConfig()

options = {
"num_keep": configLLM.num_keep,
"seed": configLLM.seed,
"num_predict": configLLM.num_predict,
"top_k": configLLM.top_k,
"top_p": configLLM.top_p,
"tfs_z": configLLM.tfs_z,
"typical_p": configLLM.typical_p,
"repeat_last_n": configLLM.repeat_last_n,
"temperature": configLLM.temperature,
"repeat_penalty": configLLM.repeat_penalty,
"presence_penalty": configLLM.presence_penalty,
"frequency_penalty": configLLM.frequency_penalty,
"mirostat": configLLM.mirostat,
"mirostat_tau": configLLM.mirostat_tau,
"mirostat_eta": configLLM.mirostat_eta,
"penalize_newline": configLLM.penalize_newline,
#"stop": config.stop,
"numa": configLLM.numa,
"num_ctx": configLLM.num_ctx,
"num_batch": configLLM.num_batch,
"num_gpu": configLLM.num_gpu,
"main_gpu": configLLM.main_gpu,
"low_vram": configLLM.low_vram,
"f16_kv": configLLM.f16_kv,
"vocab_only": configLLM.vocab_only,
"use_mmap": configLLM.use_mmap,
"use_mlock": configLLM.use_mlock,
"num_thread": configLLM.num_thread
}

### 2) Указываем инфо для данных

In [58]:
BENCHMARKS_INFO = {'sberquad': 'v1', 'squadv2': 'v1'}
SAVE_LOGFILE = '/home/aisummer/ssemenov_workspace/nlp-service/benchmark/logs/trial1.json'
banchmark_paths = {}
for name, version in BENCHMARKS_INFO.items():
    banchmark_paths[name] = {
        'table': f"../data/{name}/tables/{version}/benchmark.csv",
        'dense_db': f"../data/{name}/dbs/{version}/{DBS_DIR_DENSE_VECTORDB_NAME}",
        'sparse_db':  f"../data/{name}/dbs/{version}/{DBS_DIR_SPARSE_VECTORDB_NAME}"
    }

### 3) Загрузка датасета

In [59]:
# загрузить benchmark-датасет
benchmarks_df = {}
for name, bench_path in banchmark_paths.items():
    benchmarks_df[name] = pd.read_csv(banchmark_paths[name]['table'], sep=';')

# инициализировать класс с метриками
base_dir = '/home/aisummer/ssemenov_workspace/nlp-service'
metrics = ReaderMetrics(base_dir)

# logging
logger = Logger(False)
log = logger.get_logger(__name__)

Loading Meteor...
Loading ExactMatch


In [53]:
benchmarks_df['squadv2']['contexts'][1]

"['В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.']"

In [115]:
def get_relevant_ans(df: dict, reader: SummarizerModule) -> list:
    relevant_ans = []
    question = df['question']
    contexts = df['contexts']

    for i in range(20):
        reader.create_answer(config, configLLM, question[i], contexts[i].strip('[]').strip("'"))
        relevant_ans.append(config.answer)
    return relevant_ans

In [122]:

benchmarks_score = []

predicted_sberquad = get_relevant_ans(benchmarks_df['sberquad'], reader)
targets_sberquad = benchmarks_df['sberquad']['answer'].to_list()

# predicted_squadv2 = get_relevant_ans(benchmarks_df['squadv2'], reader)
# targets_squadv2 = benchmarks_df['squadv2']['answer'].to_list()

def metric(benchmarks_score:list[dict], predicted: list[str], targets: list[str], name:str) -> None:
    
    length = len(predicted)
    # print('Question: ', benchmarks_df['sberquad']['question'][i])
    # print('Context: ', benchmarks_df['sberquad']['contexts'][i].strip('[]').strip("'"))
    # print('Predicted: ', predicted[i])
    # print('Target: ', targets[i])

    rouge_score = metrics.rouge(predicted, targets[:length])
    print("rouge: ", rouge_score)

    bleu_score = metrics.bleu(predicted, targets[:length])
    print("bleu: ", bleu_score)

    meteor = metrics.meteor(predicted, targets[:length])
    print("meteor: ", meteor)

    exact_match = metrics.exact_match(predicted, targets[:length])
    print("exact_match: ", exact_match)

    # посчитать метрики 
    score = {
        'name': name,
        'rouge': rouge_score,
        'bleu': bleu_score,
        'meteor': meteor,
        'exact_match': exact_match
    }

    benchmarks_score.append(score)



metric(benchmarks_score, predicted_sberquad, targets_sberquad,'sberquad')
# metric(benchmarks_score, predicted_squadv2, targets_squadv2)











rouge:  0.2
bleu:  0.05811
meteor:  0.44569
exact_match:  0.0


In [123]:
benchmarks_score

[{'name': 'sberquad',
  'rouge': 0.2,
  'bleu': 0.05811,
  'meteor': 0.44569,
  'exact_match': 0.0}]

In [8]:
 # сохранить результат
if os.path.exists(SAVE_LOGFILE):
    print("Файл существует!")
    raise ValueError

log_data = {'info': BENCHMARKS_INFO, 'params': options.items}

with open(SAVE_LOGFILE, 'w', encoding='utf-8') as fd:
    fd.write(json.dumps(log_data, indent=1))

FileNotFoundError: [Errno 2] No such file or directory: './logs/trial1.json'